In [ ]:
AAA=[["Lonen",
"Minimumuur- en maandlonen","Jongerenlonen (niet studentenlonen)","Studentenlonen","Gewaarborgd gemiddeld minimumloon of -inkomen (ggmmi cao 43)","Variabel loon","Stukloon","Huisarbeid","Reële lonen","Indexeringsbepalingen","Loonsverhogingen","Openingsclausules","Loonnorm","Loon - administratieve en uitbetalingsmodaliteiten"]
,[
"Functieclassificatie","Functieclassificatie","Evaluatie van de werknemer - procedure","Loonkloof"],

["Arbeidsduur","Wekelijkse arbeidsduur of op jaarbasis","Dagelijkse arbeidsduur","Arbeidsduurvermindering","Betaalde verlofdag en feestdag","Anciënniteitsverlof (andere dan eindeloopbaandagen)","Bijkomende verlofdagen volgens leeftijd/eindeloopbaandagen","Schenken van conventioneel verlof","Arbeidsduurmodaliteiten","Rusttijden / pauzes","Wachttijd"],
["Arbeidsduurflexibiliteit en overuren - arbeidsorganisatie","Overuren - vergoeding en/of recuperatie","Overuren - quotum","Loopbaansparen",
"Telewerk","Tijdelijke arbeid (geen uitzendarbeid)","Uitzendarbeid","Deeltijdse arbeid","Loopbaanonderbreking","Tijdskrediet/loopbaanvermindering","Landingsbanen","Klein verlet","Verlof om dwingende reden","Ouderschapsverlof","Verlof om persoonlijke redenen"]
,["Aanwervingen en opleidingen","Aanwerving","Opleiding (excl. syndicale vorming)","Opleiding voor jongeren","Economische werkloosheid bedienden"],

["Werkgelegenheidsmaatregelen en doelgroepen","Maatregel voor onderneming in moeilijkheden/herstructurering-niet swt","Ontslagvergoeding bij sluiting of collectief ontslag / afscheidspremie","Oudere werknemers:anciënniteitsverlof,-toeslag,-premie/afscheidspremie","Oudere werknemers: eindeloopbaandagen en verlofdagen volgens leeftijd","Oudere werknemers: landingsbanen","Oudere werknemers: leeftijdsgebonden toeslagen",
"Oudere werknemers: werkgelegenheidsmaatregelen","Risicogroepen"],

[
"Veiligheid, gezondheid en welzijn",
"Sociale maribel",
"Veiligheid op het werk",
"Welzijn op het werk",
"Discriminatie",
"Ziekte/ongeval/overlijden" ],[
"Beëindiging van de arbeidsovereenkomst en werkloosheid",
"Economische werkloosheid bedienden",
"Werkloosheid (andere dan economische werkloosheid voor bedienden)",
"Brugpensioen",
"Stelsel van werkloosheid met bedrijfstoeslag (swt)",
"Opzegging/ontslag - organisatie en procedure",
"Opzegging/ontslag - vergoeding",
"Opzegging/ontslag - werkgelegenheidsmaatregelen" ]
,
[
"Syndicale vertegenwoordiging en premie",
"Ontslag beschermde werknemers",
"Syndicale afvaardiging",
"Syndicale vorming",
"Syndicale premie"]
,
[
"Sociale fondsen",
"Fondsen voor bestaanszekerheid",
"Sociale fondsen, andere dan fondsen voor bestaanszekerheid"]
,[

"Werking comité/onderneming",
"Werking paritair comité",
"Arbeidsreglement",
"Conventionele overlegorganen (andere dan sa, or, cpbw)",
"Syndicale afvaardiging",
"Ondernemingsraad",
"Comité voor preventie en bescherming op het werk",
"Bijzondere cao bij wijziging ressort pc"]
,[
"Diverse onderwerpen: overgang personeel, harmonisering, detachering",
"Overgang van personeel naar een andere onderneming",
"Harmonisering statuut arbeiders/bedienden",
"Detachering"]
]
import pandas as pd
df = pd.DataFrame(AAA)
for x in range(len(AAA)):
    for y in range(len(AAA[x])):
        df[x][y]=str(AAA[x][y])
df.to_csv('../csv/CLA_themes.csv',sep=';')
import pandas as pd
import random

df_m=pd.read_csv("../data/CLA_meta_from_2018.csv") 
df = pd.read_csv("../csv/NL_doc_paragraphs_extra_mean.csv")
df_meta = pd.read_csv("../csv/CLA_meta_keywords.csv")
df_themes = pd.read_csv("../csv/CLA_themes.csv",sep=';')

In [ ]:
# Add custom stopwords
from nltk.corpus import stopwords
stopwords = stopwords.words('dutch')

custom_stopwords=['waarvoor','wegens','sommige','betreffende']
stopwords.extend(custom_stopwords)
print (stopwords)

In [ ]:
# This script creates a dataframe with the high ranked keywords per document

from rake_nltk import Metric, Rake
rake_nltk_var = Rake(language='dutch',stopwords=stopwords,include_repeated_phrases=False,ranking_metric=Metric.WORD_DEGREE,max_length=2)
import re
from os import listdir

def make_targets_nlp():
    input_path='../processed_data/NL'
    file_list=listdir(input_path)

    doc_list=[]
    for file in file_list:
        document_id=file[3:-4]
        doc_list.append(document_id) 

    df_target_full=pd.DataFrame()
    for document_id in doc_list:
        # Get the themes_nl metadata for document_id in df_m
        text=df_m.loc[df_m['filename'].str.contains(document_id),'themes_nl'].values.astype(str).tolist()
    
        # Check if value is filled and extract keywords
        if pd.isnull(df_m.loc[df_m['filename'].str.contains(document_id),'themes_nl'].values)!=True:
            text = re.sub("\)\\\\","",text[0])
            text = re.sub("\)\.","",text)
            text = re.sub(" \)","",text)
            rake_nltk_var.extract_keywords_from_text(text)
            keyword_extracted = rake_nltk_var.get_ranked_phrases()
    

        meta_theme=df_meta.loc[df_meta['filename'].str.contains(document_id),'themes_nl'].values[0]
    
        doc_keywords=[document_id] 
        targets=set(keyword_extracted)
        doc_keywords=[document_id]

        for x in targets:
            doc_keywords.append(x)


        df_target=pd.DataFrame(doc_keywords)
        df_target=df_target.transpose()

        df_target_full=pd.concat([df_target_full,df_target],ignore_index=True)

    df_target_full.rename(columns = {0:'document_id'}, inplace = True)
    df_target_full.reset_index(drop=True,inplace=True)
    df_target_full.to_csv('../csv/document_targets.csv', sep=";")

In [ ]:
# Execute the script

#make_targets_nlp()

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

def clean_text(text):
  text = text.lower()
  text = re.sub("[^a-zA-Z\'\-éôëè]", " ", text) 
  return " ".join(word_tokenize(text)[:])


In [ ]:
# Make a dataframe for the model 
# and write it to document_targets.csv

import os

def make_nlp_model_df():
    input_path=os.path.join('..','processed_data','NL')
    file_list=listdir(input_path)
    doc_list=[]

    for file in file_list:
        document_id=file[3:-4]
        doc_list.append(document_id) 

    df_nlp_model=pd.read_csv('../csv/document_targets.csv',sep=';')

    for document_id in doc_list:
        with open(os.path.join(input_path,f'NL_{document_id}.txt'),encoding="utf-8") as f:
            text = f.read() 
            df_nlp_model.loc[df_nlp_model['document_id']==document_id,'text']=clean_text(text)

    df_nlp_model.reset_index(drop=True,inplace=True)
    df_nlp_model.to_csv('../csv/document_targets.csv', sep=";", index=False)

In [ ]:
#make_nlp_model_df()

In [ ]:
df_nlp_model=pd.read_csv('../csv/document_targets.csv',sep=';')
df_nlp_model.head()

In [ ]:
#Check targets to themes from website

df_meta.head()

In [ ]:

def make_targets():
    input_path='../processed_data/NL'
    file_list=listdir(input_path)

    doc_list=[]
    for file in file_list:
        document_id=file[3:-4]
        doc_list.append(document_id) 

    df_target_full=pd.DataFrame()
    for document_id in doc_list:
        # Get the themes_nl metadata for document_id in df_m
        text=df_m.loc[df_m['filename'].str.contains(document_id),'themes_nl'].values.astype(str).tolist()
    
        # Check if value is filled and extract keywords
        if pd.isnull(df_m.loc[df_m['filename'].str.contains(document_id),'themes_nl'].values)!=True:
            text = re.sub("\)\\\\","",text[0])
            text = re.sub("\)\.","",text)
            text = re.sub(" \)","",text)
            #rake_nltk_var.extract_keywords_from_text(text)
            #keyword_extracted = rake_nltk_var.get_ranked_phrases()
            #print (text)

        meta_theme=df_meta.loc[df_meta['filename'].str.contains(document_id),'themes_nl'].values[0]
        #print (meta_theme)    
        targets=[]

        text=str(text)
        if len(text)>0:
            text=re.sub('\\\\','@@',text)
            while len(text)>0:
                stop=text.find('@@')
                #print (stop)
                if stop == -1:
                    stop = len (text)
                target=text[0:stop]
                text=text[stop+2:]
                targets.append(target)

        targets=set(targets)
        doc_targets=[document_id]

        for x in targets:
            doc_targets.append(x)
                         
        print (doc_targets)
        df_target=pd.DataFrame(doc_targets)
        df_target=df_target.transpose()

        df_target_full=pd.concat([df_target_full,df_target],ignore_index=True)

    df_target_full.rename(columns = {0:'document_id'}, inplace = True)
    df_target_full.reset_index(drop=True,inplace=True)
    df_target_full.to_csv('../csv/NL_document_full_targets.csv', sep=";")

#make_targets()

In [ ]:
# Make a dataframe for the model 
# and write it to document_targets.csv

import os

def make_model_df():
    input_path=os.path.join('..','processed_data','NL')
    file_list=listdir(input_path)
    doc_list=[]

    for file in file_list:
        document_id=file[3:-4]
        doc_list.append(document_id) 

    df_model=pd.read_csv('../csv/NL_document_full_targets.csv',sep=';')

    for document_id in doc_list:
        with open(os.path.join(input_path,f'NL_{document_id}.txt'),encoding="utf-8") as f:
            text = f.read() 
            df_model.loc[df_nlp_model['document_id']==document_id,'text']=clean_text(text)

    df_model.reset_index(drop=True,inplace=True)
    df_model.to_csv('../csv/NL_document_full_targets.csv', sep=";", index=False)

#make_model_df()

In [ ]:
df_model=pd.read_csv('../csv/NL_document_full_targets.csv',sep=';')

X = df_model["text"].tolist()
y = pd.get_dummies(df_model['1'])

mapping = {i: name for i, name in enumerate(y.columns)}

#print (mapping)

In [ ]:
print (len(y))

In [ ]:
from sklearn.model_selection import train_test_split
# Split Train and Validation data

#print (len(X))
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#print ('X_train_val',len(X_train_val))
#print ('X_test',len(X_test))

print (len(X))
# Keep some data for inference (testing)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)
print ('X_train',len(X_train))
print ('X_test',len(X_test))
print ('X_val',len(X_val))

In [ ]:
print (y_val)

In [ ]:
import tensorflow as tf
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, max_length=500, truncation=True, padding=True)
val_encodings = tokenizer(X_val, max_length=500, truncation=True, padding=True)
test_encodings = tokenizer(X_test, max_length=500, truncation=True, padding=True)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(2)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(2)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(2)

In [ ]:
print (len(set(y)))

In [ ]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(y)))

In [ ]:
OPTIMIZER =  tf.keras.optimizers.Adam(learning_rate=1)
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
METRICS = ['accuracy']

model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
model.summary()


In [ ]:
labels=set(y)
BATCH_SIZE = 8
#model.fit(dict(train_dataset.batch(BATCH_SIZE)),dict(validation_data=val_dataset.batch(BATCH_SIZE)))
#model.fit(train_dataset.batch(BATCH_SIZE))
#model.fit(dict(train_encodings), y_train.to_numpy())

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
BATCH_SIZE = 8
EPOCHS = 5
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)